In [1]:
from openpyxl import load_workbook, Workbook

In [4]:
class __excel:
    def __init__(self, name):
        self.i = 0
        self.name = name
        self.filename = self.name + ".xlsx"
        workbook = Workbook()
        workbook.save(filename=self.filename)
    def get_sheets(self):
        workbook = load_workbook(filename=self.filename)
        return workbook.sheetnames
    def add_sheets(self, *col_name):
        workbook = load_workbook(filename=self.filename)
        for value in col_name:
            if value not in workbook.sheetnames:
                col_name = workbook.create_sheet(value, self.i)
                self.i += 1
            else:
                print(f"{value} already exists in {self.name}. Ignoring column creation!")
        if "Sheet" in workbook.sheetnames:
            del workbook["Sheet"]
        workbook.save(filename=self.filename)
    def write(self, sheet, key, index, value):
        workbook = load_workbook(filename=self.filename)
        ws = workbook[f"{sheet}"]
        ws[f"{key}{index}"] = value
        workbook.save(filename=self.filename)

In [5]:
book1 = __excel("Test")
book1.add_sheets("Sheet1","Sheet2","Sheet3")

In [ ]:
IP_List = ["192.168.0.1","192.168.0.2","192.168.0.3",]
Hostname_List = ["Hostname1","Hostname2","Hostname3",]
Interface_List = ["Vlan1","Vlan2","Vlan3",]
Domain_List = ["Test1.com","Test2.com","Test3.com",]

In [ ]:
i = 1
for a,b,c,d in zip(Interface_List, IP_List, Hostname_List, Domain_List):
    book1.write("Sheet1","A",f"{i}",f"{a}")
    book1.write("Sheet1","B",f"{i}",f"{b}")
    book1.write("Sheet1","C",f"{i}",f"{c}")
    book1.write("Sheet1","D",f"{i}",f"{d}")
    i += 1

In [6]:
def prep_workbook():
    book1 = __excel("Test")
    book1.add_sheets("Sheet1","Sheet2","Sheet3")
    book1.write("Sheet1","A","1","Interface",)
    book1.write("Sheet1","B","1","IP Address",)
    book1.write("Sheet1","C","1","Hostname",)
    book1.write("Sheet1","D","1","Domain Name",)

In [7]:
prep_workbook()

In [ ]:
output_log = open("Output Log.txt", "a")
output_log.write("This is a message\n")
output_log.close()

In [35]:
from datetime import datetime

In [40]:
def output_log(message):
    dateTimeObj = datetime.now()
    output_log = open("Output Log.txt", "a")
    output_log.write(f"{dateTimeObj} - {message}")
    output_log.write("\n")
    output_log.close()

In [43]:
output_log("This is a message")

In [11]:
list = (("VLAN1","192.168.0.1","host1","domain.com"),("VLAN2","192.168.0.2","host2","domain.com"),("VLAN3","192.168.0.3","host3","domain.com"),("VLAN4","192.168.0.4","host4","domain.com"))

In [12]:
list

(('VLAN1', '192.168.0.1', 'host1', 'domain.com'),
 ('VLAN2', '192.168.0.2', 'host2', 'domain.com'),
 ('VLAN3', '192.168.0.3', 'host3', 'domain.com'),
 ('VLAN4', '192.168.0.4', 'host4', 'domain.com'))

In [17]:
a,b,c,d = list

In [33]:
i = 2
for a,b,c,d in list:
    book1.write("Sheet1","A",f"{i}",f"{a}",)
    book1.write("Sheet1","B",f"{i}",f"{b}",)
    book1.write("Sheet1","C",f"{i}",f"{c}",)
    book1.write("Sheet1","D",f"{i}",f"{d}",)
    i += 1